# Conduct Complex Analysis with Pro Mode

> #################################################################################
>
> **Note:** Pro mode is currently available only for `document` data.  
> [Supported file types](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/service-limits#document-and-text): pdf, tiff, jpg, jpeg, png, bmp, heif
>
> #################################################################################

This notebook demonstrates how to use **Pro mode** in Azure AI Content Understanding to enhance your analyzer with multiple inputs and optional reference data. Pro mode is designed for advanced use cases, particularly those requiring multi-step reasoning and complex decision-making (for example, identifying inconsistencies, drawing inferences, and making sophisticated decisions). Pro mode allows input from multiple content files and includes the option to provide reference data at analyzer creation time.

In this walkthrough, you'll learn how to:
1. Create an analyzer with a schema and reference data.
2. Analyze your files using Pro mode.

For more details on Pro mode, see the [Azure AI Content Understanding: Standard and Pro Modes](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/concepts/standard-pro-modes) documentation.

## Prerequisites
1. Ensure the Azure AI service is configured by following the [setup steps](../README.md#configure-azure-ai-service-resource).
2. If using reference documents, please follow [Set env for reference doc](../docs/set_env_for_training_data_and_reference_doc.md) to configure reference document environment variables in the [.env](./.env) file.
    - You can set `REFERENCE_DOC_SAS_URL` directly with the SAS URL for your Azure Blob container.
    - Alternatively, set both `REFERENCE_DOC_STORAGE_ACCOUNT_NAME` and `REFERENCE_DOC_CONTAINER_NAME` so that the SAS URL can be generated automatically during a later step.
    - Also, set `REFERENCE_DOC_PATH` to specify the folder path within the container where reference documents will be uploaded.
    > ⚠️ Note: Reference documents are optional in Pro mode. You can run Pro mode using only input documents. For example, the service can reason across two or more input files without any reference data.
3. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Analyzer Template and Local Files Setup
- **analyzer_template**: In this sample, we define an analyzer template for invoice-contract verification.
- **input_docs**: You can have multiple input document files in one folder or specify a single document file location.
- **reference_docs (Optional)**: During analyzer creation, you can provide documents that aid in providing contextual information for the analyzer to reference during inference. OCR results will be extracted from these files if needed, a reference JSONL file will be generated, and these files will be uploaded to a designated Azure Blob Storage container.

> For example, if you're analyzing invoices to ensure their consistency with a contractual agreement, you can supply the invoice and other relevant documents (e.g., a purchase order) as inputs, and provide the contract files as reference data. The service applies reasoning to validate the input documents against your schema, which might include identifying discrepancies to flag for further review.

In [ ]:
# Define paths for analyzer template, input documents, and reference documents
analyzer_template = "../analyzer_templates/invoice_contract_verification_pro_mode.json"
input_docs = "../data/field_extraction_pro_mode/invoice_contract_verification/input_docs"

# NOTE: Reference documents are optional in Pro mode. Comment out the line below if not using reference documents.
reference_docs = "../data/field_extraction_pro_mode/invoice_contract_verification/reference_docs"

> Let's examine the Pro mode analyzer template.

In [ ]:
import json
with open(analyzer_template, "r") as file:
    print(json.dumps(json.load(file), indent=2))

> In the analyzer, the field `"mode"` must be set to `"pro"`. The defined field `"PaymentTermsInconsistencies"` is a `"generate"` field designed to reason about inconsistencies. It can utilize the referenced documents uploaded in [reference docs](../data/field_extraction_pro_mode/invoice_contract_verification/reference_docs).

## Create Azure Content Understanding Client
> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing client functions. Before the official release of the Content Understanding SDK, consider it a lightweight SDK. Fill in the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, and **AZURE_AI_API_KEY** with your Azure AI Service details.

> ⚠️ Important:
You must update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample may not run correctly.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is recommended and safer for production environments.

In [ ]:
import logging
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Import utility package from python samples root directory
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based auth or subscription key; only one is required
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set it in the ".env" file if not using token auth
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment this line if using subscription key
    # subscription_key=AZURE_AI_API_KEY,
    x_ms_useragent="azure-ai-content-understanding-python/pro_mode", # This header is used for sample usage telemetry; please comment out this line if you want to opt out.
)

## Prepare Reference Data
In this step, we will:
- Use `REFERENCE_DOC_PATH` and SAS URL related environment variables that were set in the Prerequisites.
- Attempt to obtain the SAS URL from the `REFERENCE_DOC_SAS_URL` environment variable.
  If this is not set, the SAS URL will be generated automatically using the `REFERENCE_DOC_STORAGE_ACCOUNT_NAME` and `REFERENCE_DOC_CONTAINER_NAME` environment variables.
- Use Azure AI service to extract OCR results from reference documents if needed.
- Generate a reference `.jsonl` file.
- Upload these files to the designated Azure Blob Storage.


In [ ]:
# Load reference storage configuration from environment
reference_doc_path = os.getenv("REFERENCE_DOC_PATH")

reference_doc_sas_url = os.getenv("REFERENCE_DOC_SAS_URL")
if not reference_doc_sas_url:
    REFERENCE_DOC_STORAGE_ACCOUNT_NAME = os.getenv("REFERENCE_DOC_STORAGE_ACCOUNT_NAME")
    REFERENCE_DOC_CONTAINER_NAME = os.getenv("REFERENCE_DOC_CONTAINER_NAME")
    if REFERENCE_DOC_STORAGE_ACCOUNT_NAME and REFERENCE_DOC_CONTAINER_NAME:
        from azure.storage.blob import ContainerSasPermissions
        # We require "Write" permission to upload, modify, or append blobs
        reference_doc_sas_url = AzureContentUnderstandingClient.generate_temp_container_sas_url(
            account_name=REFERENCE_DOC_STORAGE_ACCOUNT_NAME,
            container_name=REFERENCE_DOC_CONTAINER_NAME,
            permissions=ContainerSasPermissions(read=True, write=True, list=True),
            expiry_hours=1,
        )

> ⚠️ Note: Reference documents are optional in Pro mode. You can run Pro mode using only input documents. For example, the service can reason across two or more input files without any reference data. To skip reference document preparation, please skip or comment out the code in the following section.

In [ ]:
# Set skip_analyze to True if you already have OCR results for the documents in the reference_docs folder
# Please name the OCR result files with the same name as the original document filenames including extension, and add the suffix ".result.json"
# For example, if the original document is "invoice.pdf", the OCR result file should be named "invoice.pdf.result.json"
# NOTE: Please comment out the following line if you do not have any reference documents.
await client.generate_knowledge_base_on_blob(reference_docs, reference_doc_sas_url, reference_doc_path, skip_analyze=False)

## Create Analyzer with Defined Schema for Pro Mode
Before creating the analyzer, assign a relevant name to the constant **CUSTOM_ANALYZER_ID** for your task. Here, we generate a unique suffix so this cell can be run multiple times to create different analyzers.

We use **reference_doc_sas_url** and **reference_doc_path** configured in the [.env](./.env) file and utilized in the previous step.

In [ ]:
import uuid
CUSTOM_ANALYZER_ID = "pro-mode-sample-" + str(uuid.uuid4())

response = client.begin_create_analyzer(
    CUSTOM_ANALYZER_ID,
    analyzer_template_path=analyzer_template,
    pro_mode_reference_docs_storage_container_sas_url=reference_doc_sas_url,
    pro_mode_reference_docs_storage_container_path_prefix=reference_doc_path,
)
result = client.poll_result(response)
if result is not None and "status" in result and result["status"] == "Succeeded":
    logging.info(f"Analyzer details for {result['result']['analyzerId']}")
    logging.info(json.dumps(result, indent=2))
else:
    logging.warning(
        "An issue was encountered when trying to create the analyzer. "
        "Please double-check your deployment and configurations for potential problems."
    )

## Use Created Analyzer to Analyze Input Documents
After the analyzer is successfully created, it can be used to analyze input files.
> NOTE: Pro mode performs multi-step reasoning and may require more time for analysis.

In [ ]:
from IPython.display import FileLink, display

response = client.begin_analyze(CUSTOM_ANALYZER_ID, file_location=input_docs)
result_json = client.poll_result(response, timeout_seconds=600)  # Use a longer timeout for Pro mode

# Create output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, f"{CUSTOM_ANALYZER_ID}_result.json")
with open(output_path, "w", encoding="utf-8") as file:
    json.dump(result_json, file, indent=2)

logging.info(f"Full analyzer result saved to: {output_path}")
display(FileLink(output_path))

> Let's review the extracted fields with Pro mode.

In [ ]:
fields = result_json["result"]["contents"][0]["fields"]
print(json.dumps(fields, indent=2))

> As seen in the `PaymentTermsInconsistencies` field, for example, the purchase contract details the payment terms agreed upon prior to the service. However, the implied payment terms on the invoice conflict with these. Pro mode identified the corresponding contract from the reference documents and analyzed it alongside the invoice to discover this inconsistency.

## Delete Existing Analyzer in Content Understanding Service
This step is optional. It removes the test analyzer from your service to prevent it from persisting. Without deletion, the analyzer will remain for potential reuse.

In [ ]:
client.delete_analyzer(CUSTOM_ANALYZER_ID)

## Bonus Sample
This sample demonstrates how Pro mode supports multi-document input and advanced reasoning. Unlike Document Standard Mode, which processes one document at a time, Pro mode can analyze multiple documents within a single analysis call. The service not only processes each document independently but also cross-references documents to perform reasoning across them, enabling deeper insights and validation.

### First, Set Up Variables for the Second Sample

In [ ]:
# Define paths for analyzer template, input documents, and reference documents of the second sample
analyzer_template_2 = "../analyzer_templates/insurance_claims_review_pro_mode.json"
input_docs_2 = "../data/field_extraction_pro_mode/insurance_claims_review/input_docs"
reference_docs_2 = "../data/field_extraction_pro_mode/insurance_claims_review/reference_docs"

# Load reference storage configuration from environment
reference_doc_path_2 = os.getenv("REFERENCE_DOC_PATH").rstrip("/") + "_2/"  # NOTE: Use a different path for the second sample
CUSTOM_ANALYZER_ID_2 = "pro-mode-sample-" + str(uuid.uuid4())

### Generate Knowledge Base for the Second Sample
Upload [reference documents](../data/field_extraction_pro_mode/insurance_claims_review/reference_docs/) with existing OCR results for the second sample. These documents contain driver coverage policies useful for reviewing insurance claims.

In [ ]:
logging.info("Start generating knowledge base for the second sample...")
# Reuse the same blob container
await client.generate_knowledge_base_on_blob(reference_docs_2, reference_doc_sas_url, reference_doc_path_2, skip_analyze=True)

### Create Analyzer for the Second Sample
Reuse the existing AzureContentUnderstandingClient.

In [ ]:
response = client.begin_create_analyzer(
    CUSTOM_ANALYZER_ID_2,
    analyzer_template_path=analyzer_template_2,
    pro_mode_reference_docs_storage_container_sas_url=reference_doc_sas_url,
    pro_mode_reference_docs_storage_container_path_prefix=reference_doc_path_2,
)
result = client.poll_result(response)
if result is not None and "status" in result and result["status"] == "Succeeded":
    logging.info(f"Analyzer details for {result['result']['analyzerId']}")
    logging.info(json.dumps(result, indent=2))
else:
    logging.warning(
        "An issue was encountered when trying to create the analyzer. "
        "Please double-check your deployment and configurations for potential problems."
    )

### Analyze Multiple Input Documents with the Second Analyzer
The [input_docs_2](../data/field_extraction_pro_mode/insurance_claims_review/input_docs/) folder contains two PDFs as input: a car accident report and a repair estimate.

The first document includes details such as the car’s license plate number, vehicle model, and incident information.
The second document provides a breakdown of the estimated repair costs.

Because of the complexity of this multi-document scenario, analysis may take several minutes.

In [ ]:
logging.info("Start analyzing input documents for the second sample...")
response = client.begin_analyze(CUSTOM_ANALYZER_ID_2, file_location=input_docs_2)
result_json = client.poll_result(response, timeout_seconds=600)  # Use a longer timeout for Pro mode

# Save the result to a JSON file
# Create output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"{CUSTOM_ANALYZER_ID_2}_result.json")
with open(output_path, "w", encoding="utf-8") as file:
    json.dump(result_json, file, indent=2)

logging.info(f"Full analyzer result saved to: {output_path}")
display(FileLink(output_path))

### Review Analyze Result

In [ ]:
result_json["result"]["contents"][0]["fields"]

### Examine `LineItemCorroboration` Field in Detail

> The `ReportingOfficer` field is only present in the car accident report, while fields such as `VIN` come exclusively from the repair estimate document. This demonstrates how information is extracted from both documents to generate a single unified result.
> 
> Multiple input documents are combined to produce one consolidated output. This is a single-analysis result, unlike a batch model where N input documents yield N outputs.

In [ ]:
fields = result_json["result"]["contents"][0]["fields"]["LineItemCorroboration"]
print(json.dumps(fields, indent=2))

> Within the `LineItemCorroboration` field, each line item from the *repair estimate document* is extracted along with its claim status and evidence. Items not covered by the policy, such as a Starbucks drink and hotel stay, are marked as suspicious. Valid damage repairs supported by the claim documents and permitted under the policy are confirmed.

### [Optional] Delete the Analyzer for the Second Sample After Use

In [ ]:
client.delete_analyzer(CUSTOM_ANALYZER_ID_2)